In [340]:
import requests, re
from bs4 import BeautifulSoup as bs
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import time, random
from urllib.request import urlopen
import urllib.parse
import numpy as np
from IPython.display import display
import sys,csv, pickle

In [357]:
fig_path = '/Users/samuelblass/Desktop/Project/3. Business/Figures/'

In [8]:
def get_soup(url): # Retrieves soup from website using py requests module and 'person' headers
    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:95.0) Gecko/20100101 Firefox/95.0',
        'Accept-Language': 'en-US,en;q=0.5',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8',
        'Accept-Encoding': 'gzip, deflate, br',
        'Referer': 'https://duckduckgo.com/'
        }
    response = requests.get(url, headers=headers)
    status = response.status_code
    if status == 200:
      page = response.text
      soup = bs(page)
    else:
      print(f"Oops! Received status code {status}")
    return soup

In [328]:
def get_soup_driver(base_url): # Retrieves soup from website using Selenium
#     driver = webdriver.Chrome()
    driver.get(base_url)
    time.sleep(10 + random.uniform(-1,1))
    r = driver.page_source
    
#     driver.quit()
    return bs(r, 'html.parser')  

In [329]:
def get_zip_soup(city, state):
    city = urllib.parse.quote(city, safe='~')
    base_url = f'https://www.zip-codes.com/search.asp?fld-city={city}&fld-state={state}'
    
    return get_soup_driver(base_url)

In [124]:
driver = webdriver.Chrome()
driver.get("http://www.proximityone.com/city_daytime_population.htm")

# get table -- first wait for table to fully load
WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH, "//*[@id='aw36-scroll-box']")))
table = driver.find_element_by_xpath("//*[@id='aw36-scroll-box']")

get_number = 0
while True:
    count = get_number
    rows = driver.find_elements_by_class_name("aw-grid-row")
    driver.execute_script("arguments[0].scrollIntoView();", rows[-1])  # scroll to last row
    get_number = len(rows)
    print(get_number)
    time.sleep(1)
    if get_number == count:
        break


14
14


In [114]:
# toScroll = 
driver.execute_script("arguments[0].scrollBy(0, 180)", table);

In [266]:
results = {}
headers = driver.find_elements_by_class_name("aw-grid-header")
for h in headers:
    results[h.text] = ['']
print(results)

{'Rank': [''], 'Area Name': [''], 'St1': [''], 'Size': [''], 'St2': [''], 'PlcCode': [''], 'Population': [''], 'WorkersWP': [''], 'WorkersLP': [''], 'Daytime': [''], 'Daytime1': [''], '%Daytime1': [''], 'WorkersLW': [''], '%WorkersLW': [''], 'E-R Ratio': [''], '': ['']}


In [267]:
dict_keys = list(results.keys())
f = open("Figures/output.txt", "w")
counter = 0
stop = False
while not stop:
#     print(results['Area Name'])

    counter += 1
    rows = driver.find_elements_by_class_name("aw-grid-row")[:-2]
    for row in rows:
        data = row.text.split('\n')
#         print(data)
        for i, datum in enumerate(data):
            if datum != '':
                results[dict_keys[i]] += [datum]
            if datum == 'Lake Buena Vista': stop = True
        f.write(str(results))
    driver.execute_script("arguments[0].scrollBy(0, 180)", table)
#     if counter == 2: break
    time.sleep(2)
f.close()                    

KeyboardInterrupt: 

In [243]:
results['Rank']

['',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 '10',
 '11',
 '12',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '18',
 '19',
 '20',
 '21',
 '22']

In [269]:
del results['']

In [270]:
for key in results:
    print(len(results[key]))

8689
8689
8689
8689
8689
8689
8689
8689
8689
8689
8689
8689
8689
8689
8605


In [272]:
del results['E-R Ratio']

In [271]:
results.keys()

dict_keys(['Rank', 'Area Name', 'St1', 'Size', 'St2', 'PlcCode', 'Population', 'WorkersWP', 'WorkersLP', 'Daytime', 'Daytime1', '%Daytime1', 'WorkersLW', '%WorkersLW', 'E-R Ratio'])

In [274]:
commuter_df = pd.DataFrame(results)
commuter_df

,Rank,Area Name,St1,Size,St2,PlcCode,Population,WorkersWP,WorkersLP,Daytime,Daytime1,%Daytime1,WorkersLW,%WorkersLW
0,,,,,,,,,,,,,,
1,1,New York,NY,1,36,51000,"8,008,278","3,755,130","3,192,070","8,571,338","563,060",7.0,"2,922,206",91.5
2,2,Los Angeles,CA,1,06,44000,"3,694,820","1,622,772","1,494,895","3,822,697","127,877",3.5,"943,489",63.1
3,3,Chicago,IL,1,17,14000,"2,896,016","1,334,467","1,192,139","3,038,344","142,328",4.9,"841,329",70.6
4,4,Houston,TX,1,48,35000,"1,953,631","1,244,999","841,686","2,356,944","403,313",20.6,"681,785",81.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8684,6470,Meridian CDP,CO,9,08,50012,184,"7,905",112,"7,977","7,793",4235.3,10,8.9
8685,6471,Rock Island Arsenal CDP,IL,9,17,65104,145,"3,941",71,"4,015","3,870",2669.0,25,35.2
8686,6472,Black Hawk,CO,9,08,07025,118,"2,879",88,"2,909","2,791",2365.3,29,33.0
8687,6473,Vernon,CA,9,06,82422,91,"37,472",36,"37,527","37,436",41138.5,17,47.2


In [345]:
commuter_df = pd.read_pickle('/Users/samuelblass/Desktop/Project/3. Business/Figures/commuter_df')

In [346]:
commuter_df.drop(index=commuter_df.index[0], axis=0, inplace=True, errors='ignore')

In [347]:
commuter_df.drop_duplicates(subset = commuter_df.columns, inplace=True)

In [348]:
commuter_df.reset_index(drop=True, inplace=True)

In [349]:
commuter_df.drop(index=commuter_df.index[6472], inplace=True)

In [352]:
commuter_df.to_pickle('/Users/samuelblass/Desktop/Project/3. Business/Figures/commuter_df_cleaned')

In [337]:
commuter_df.tail()

,Rank,Area Name,St1,Size,St2,PlcCode,Population,WorkersWP,WorkersLP,Daytime,Daytime1,%Daytime1,WorkersLW,%WorkersLW
6469,6470,Meridian CDP,CO,9,08,50012,184,"7,905",112,"7,977","7,793",4235.3,10,8.9
6470,6471,Rock Island Arsenal CDP,IL,9,17,65104,145,"3,941",71,"4,015","3,870",2669.0,25,35.2
6471,6472,Black Hawk,CO,9,08,07025,118,"2,879",88,"2,909","2,791",2365.3,29,33.0
6473,6473,Vernon,CA,9,06,82422,91,"37,472",36,"37,527","37,436",41138.5,17,47.2
6474,6474,Teterboro borough,NJ,9,34,72480,18,"6,697",13,"6,702","6,684",37133.3,3,23.1


In [363]:
commuter_df_NYNJCTMD = commuter_df[(commuter_df.St1 == 'NJ') | (commuter_df.St1 == 'NY' )| (commuter_df.St1 == 'CT') | (commuter_df.St1 == 'MD')]

In [ ]:
commuter_df_NYNJCTPA = commuter_df[(commuter_df.St1 == 'NJ') | (commuter_df.St1 == 'NY' )| (commuter_df.St1 == 'CT') | (commuter_df.St1 == 'PA')]

In [364]:
commuter_df_NYNJCTMD.to_pickle('/Users/samuelblass/Desktop/Project/3. Business/Figures/commuter_df_NYNJCTMD')

In [354]:
commuter_df_MD = commuter_df[commuter_df.St1 == 'MD']
commuter_df_MD

,Rank,Area Name,St1,Size,St2,PlcCode,Population,WorkersWP,WorkersLP,Daytime,Daytime1,%Daytime1,WorkersLW,%WorkersLW
16,17,Baltimore,MD,2,24,04000,"651,154","341,998","249,373","743,779","92,625",14.2,"154,463",61.9
290,291,Columbia CDP,MD,5,24,19125,"88,254","44,965","50,175","83,044","-5,210",-5.9,"12,911",25.7
362,363,Silver Spring CDP,MD,5,24,72450,"76,540","31,077","40,668","66,949","-9,591",-12.5,"5,510",13.5
470,471,Dundalk CDP,MD,5,24,23975,"62,306","16,198","26,715","51,789","-10,517",-16.9,"5,367",20.1
533,534,Wheaton-Glenmont CDP,MD,5,24,83837,"57,694","10,314","28,323","39,685","-18,009",-31.2,"2,343",8.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6374,6375,Oakland,MD,9,24,57650,"1,930","3,636",731,"4,835","2,905",150.5,337,46.1
6381,6382,Leonardtown,MD,9,24,46475,"1,896","4,631",623,"5,904","4,008",211.4,210,33.7
6386,6387,Kensington,MD,9,24,43500,"1,873","2,712",898,"3,687","1,814",96.8,113,12.6
6421,6422,Prince Frederick CDP,MD,9,24,63950,"1,432","5,541",465,"6,508","5,076",354.5,170,36.6


In [359]:
cities_MD = commuter_df_MD[['Area Name', 'St1']]
cities_MD = cities_MD.rename(columns={'Area Name':'City', 'St1':'State'})
cities_MD.City = cities_MD.City.apply(lambda x: x.replace('CDP','').replace('(balance)',''))
cities_MD.to_csv(fig_path + 'cities_MD.csv')

In [ ]:
driver = webdriver.Chrome()


In [257]:
f = open('Figures/output.txt','w')
f.write(str(results))
f.close()

In [258]:
with open('Figures/output.txt','r') as f:
    test = f.read()

In [264]:
test = eval(test)

In [265]:
pd.DataFrame(dict(test))

,Rank,Area Name,St1,Size,St2,PlcCode,Population,WorkersWP,WorkersLP,Daytime,Daytime1,%Daytime1,WorkersLW,%WorkersLW,E-R Ratio
0,,,,,,,,,,,,,,,
1,1,New York,NY,1,36,51000,"8,008,278","3,755,130","3,192,070","8,571,338","563,060",7.0,"2,922,206",91.5,1.18
2,2,Los Angeles,CA,1,06,44000,"3,694,820","1,622,772","1,494,895","3,822,697","127,877",3.5,"943,489",63.1,1.09
3,3,Chicago,IL,1,17,14000,"2,896,016","1,334,467","1,192,139","3,038,344","142,328",4.9,"841,329",70.6,1.12
4,4,Houston,TX,1,48,35000,"1,953,631","1,244,999","841,686","2,356,944","403,313",20.6,"681,785",81.0,1.48
5,5,Philadelphia,PA,1,42,60000,"1,517,550","659,991","569,761","1,607,780","90,230",5.9,"429,608",75.4,1.16
6,6,Phoenix,AZ,1,04,55000,"1,321,045","695,712","599,592","1,417,165","96,120",7.3,"415,384",69.3,1.16
7,7,San Diego,CA,1,06,66000,"1,223,400","722,245","580,318","1,365,327","141,927",11.6,"450,898",77.7,1.24
8,8,Dallas,TX,1,48,19000,"1,188,580","764,561","537,006","1,416,135","227,555",19.1,"350,745",65.3,1.42
9,9,San Antonio,TX,1,48,65000,"1,144,646","559,424","491,435","1,212,635","67,989",5.9,"428,926",87.3,1.14


In [121]:
f.write(str(results))

43982

In [94]:
rows[0].text.split('\n')

['1',
 'New York',
 'NY',
 '1',
 '36',
 '51000',
 '8,008,278',
 '3,755,130',
 '3,192,070',
 '8,571,338',
 '563,060',
 '7.0',
 '2,922,206',
 '91.5',
 '1.18']

In [115]:
rows

[<selenium.webdriver.remote.webelement.WebElement (session="0cd96ea8e7b230ccc61b9175950f8466", element="a812d01b-a337-49ff-9985-143575177819")>,
 <selenium.webdriver.remote.webelement.WebElement (session="0cd96ea8e7b230ccc61b9175950f8466", element="939deb71-97d3-4b38-bf48-4f4d169c9358")>,
 <selenium.webdriver.remote.webelement.WebElement (session="0cd96ea8e7b230ccc61b9175950f8466", element="8e1664cc-bf30-4640-bbf4-76236ea262a6")>,
 <selenium.webdriver.remote.webelement.WebElement (session="0cd96ea8e7b230ccc61b9175950f8466", element="f7f15068-3dcc-4dfd-b1bc-ae38d7149146")>,
 <selenium.webdriver.remote.webelement.WebElement (session="0cd96ea8e7b230ccc61b9175950f8466", element="40b85d19-f0d4-4a58-b7a5-53f39c3b37bd")>,
 <selenium.webdriver.remote.webelement.WebElement (session="0cd96ea8e7b230ccc61b9175950f8466", element="ec5d9bce-fc9b-48a0-aaa4-3711e819f58d")>,
 <selenium.webdriver.remote.webelement.WebElement (session="0cd96ea8e7b230ccc61b9175950f8466", element="27d510ba-2d68-4ef4-83f6-ab

In [15]:
driver = webdriver.Chrome()
driver.get("http://www.proximityone.com/city_daytime_population.htm")

# get table -- first wait for table to fully load
WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH, "//*[@id='aw36-view-box-middle']")))
table = driver.find_element_by_xpath("//*[@id='aw36-view-box-middle']")



In [66]:
table

<selenium.webdriver.remote.webelement.WebElement (session="264080561637bd3200d9a9666243138f", element="ff2965c5-c316-407e-8cd4-be158c956e70")>

In [89]:
rows = driver.find_elements_by_class_name("aw-grid-row")

In [90]:
rows

[<selenium.webdriver.remote.webelement.WebElement (session="264080561637bd3200d9a9666243138f", element="fecfaeef-0833-40d5-8f86-29541ad27222")>,
 <selenium.webdriver.remote.webelement.WebElement (session="264080561637bd3200d9a9666243138f", element="fe55c5ce-4f03-43cb-9651-05cb852e2903")>,
 <selenium.webdriver.remote.webelement.WebElement (session="264080561637bd3200d9a9666243138f", element="d5a2fbf0-a250-40f0-86a1-b6757c97827e")>,
 <selenium.webdriver.remote.webelement.WebElement (session="264080561637bd3200d9a9666243138f", element="a57ab211-3198-4c77-b0c4-d10778fbaf70")>,
 <selenium.webdriver.remote.webelement.WebElement (session="264080561637bd3200d9a9666243138f", element="913c236a-8f75-4b3e-a881-0464293e41b6")>,
 <selenium.webdriver.remote.webelement.WebElement (session="264080561637bd3200d9a9666243138f", element="c3e7f635-0c02-44e0-bdef-5c0ff0d1466a")>,
 <selenium.webdriver.remote.webelement.WebElement (session="264080561637bd3200d9a9666243138f", element="7ac8202b-2ed4-47f5-ad2f-a0

In [73]:
for row in rows: print (row.text)


Las Colonias CDP
TX
9
48
41563
283
2,885
68
3,100
2,817
995.4
0
0.0
42.43
6465
McCook
IL
9
17
45564
254
4,780
126
4,908
4,654
1832.3
32
25.4
37.94
6466
Sauget
IL
9
17
67756
249
3,576
80
3,745
3,496
1404.0
34
42.5
44.70
6467
Dune Acres
IN
9
18
18982
213
5,959
105
6,067
5,854
2748.4
24
22.9
56.75
6468
Westlake
TX
9
48
77620
207
3,633
99
3,741
3,534
1707.2
28
28.3
36.70
6469
Crane
IN
9
18
15652
203
3,852
89
3,966
3,763
1853.7
34
38.2
43.28
6470
Meridian CDP
CO
9
08
50012
184
7,905
112
7,977
7,793
4235.3
10
8.9
70.58
6471
Rock Island Arsenal CDP
IL
9
17
65104
145
3,941
71
4,015
3,870
2669.0
25
35.2
55.51
6472
Black Hawk
CO
9
08
07025
118
2,879
88
2,909
2,791
2365.3
29
33.0
32.72
6473
Vernon
CA
9
06
82422
91
37,472
36
37,527
37,436
41138.5
17
47.2
1040.89
6474
Teterboro borough
NJ
9
34
72480
18
6,697
13
6,702
6,684
37133.3
3
23.1
515.15
6475
Lake Buena Vista
FL
9
12
37625
16
30,768
10
30,774
30,758
192237.5
4
40.0
3076.80



In [96]:
len(rows[0].text.split('\n'))

15

In [97]:
for key in results: print(key)

Rank
Area Name
St1
Size
St2
PlcCode
Population
WorkersWP
WorkersLP
Daytime
Daytime1
%Daytime1
WorkersLW
%WorkersLW
E-R Ratio



In [104]:
list(results.keys())

['Rank',
 'Area Name',
 'St1',
 'Size',
 'St2',
 'PlcCode',
 'Population',
 'WorkersWP',
 'WorkersLP',
 'Daytime',
 'Daytime1',
 '%Daytime1',
 'WorkersLW',
 '%WorkersLW',
 'E-R Ratio',
 '']

In [105]:
len(rows)

14

In [34]:
table.find_elements_by_class_name("aw-grid-row")

[<selenium.webdriver.remote.webelement.WebElement (session="88e4100af96e80e32d6ba5613c98f1f9", element="2434f9fe-df4f-4513-92a6-9da705ecb82c")>,
 <selenium.webdriver.remote.webelement.WebElement (session="88e4100af96e80e32d6ba5613c98f1f9", element="fc9dc67f-a6dc-480a-aa77-b12a9144a1b5")>,
 <selenium.webdriver.remote.webelement.WebElement (session="88e4100af96e80e32d6ba5613c98f1f9", element="c6ae6c6b-f1fb-498b-8a16-1355aca55693")>,
 <selenium.webdriver.remote.webelement.WebElement (session="88e4100af96e80e32d6ba5613c98f1f9", element="8b5c2090-d531-47de-84df-ef15453dd2f1")>,
 <selenium.webdriver.remote.webelement.WebElement (session="88e4100af96e80e32d6ba5613c98f1f9", element="946411bb-e15d-479f-b41f-cc175fdea9e1")>,
 <selenium.webdriver.remote.webelement.WebElement (session="88e4100af96e80e32d6ba5613c98f1f9", element="1bc4166c-4a54-47f0-b389-8005381ca56d")>,
 <selenium.webdriver.remote.webelement.WebElement (session="88e4100af96e80e32d6ba5613c98f1f9", element="57608133-34ae-43b7-bcc5-ba

In [33]:
table.text.split('\n')

['1',
 'New York',
 'NY',
 '1',
 '36',
 '51000',
 '8,008,278',
 '3,755,130',
 '3,192,070',
 '8,571,338',
 '563,060',
 '7.0',
 '2,922,206',
 '91.5',
 '1.18',
 '2',
 'Los Angeles',
 'CA',
 '1',
 '06',
 '44000',
 '3,694,820',
 '1,622,772',
 '1,494,895',
 '3,822,697',
 '127,877',
 '3.5',
 '943,489',
 '63.1',
 '1.09',
 '3',
 'Chicago',
 'IL',
 '1',
 '17',
 '14000',
 '2,896,016',
 '1,334,467',
 '1,192,139',
 '3,038,344',
 '142,328',
 '4.9',
 '841,329',
 '70.6',
 '1.12',
 '4',
 'Houston',
 'TX',
 '1',
 '48',
 '35000',
 '1,953,631',
 '1,244,999',
 '841,686',
 '2,356,944',
 '403,313',
 '20.6',
 '681,785',
 '81.0',
 '1.48',
 '5',
 'Philadelphia',
 'PA',
 '1',
 '42',
 '60000',
 '1,517,550',
 '659,991',
 '569,761',
 '1,607,780',
 '90,230',
 '5.9',
 '429,608',
 '75.4',
 '1.16',
 '6',
 'Phoenix',
 'AZ',
 '1',
 '04',
 '55000',
 '1,321,045',
 '695,712',
 '599,592',
 '1,417,165',
 '96,120',
 '7.3',
 '415,384',
 '69.3',
 '1.16',
 '7',
 'San Diego',
 'CA',
 '1',
 '06',
 '66000',
 '1,223,400',
 '722,245',

In [ ]:
list_results = []

In [ ]:
while True:
    list_results
    

In [87]:
f = open("output.txt", "a")
print("Hello stackoverflow!", file=f)
print("I have a question.", file=f)
f.close()

```test```